# Lista 1

O objetivo em estimar o vetor de parâmetros beta é encontrar os coeficientes que melhor descrevem a relação entre as variáveis independentes (idade, gênero, número de dependentes, fumante/não-fumante) e a variável dependente (despesas médicas). Esses coeficientes minimizam a diferença entre os valores observados e os valores preditos pelo modelo, geralmente utilizando o critério de mínimos quadrados.

In [21]:
import numpy as np
import pandas as pd

# Carregar os dados
url = "https://raw.githubusercontent.com/Cayan-Portela/ceub/main/dados/insurance_teste.csv"
data = pd.read_csv(url, sep=';')


# Substituir vírgulas por pontos e converter colunas 'bmi' e 'charges' para float
data['bmi'] = data['bmi'].str.replace(',', '.').astype(float)
data['charges'] = data['charges'].str.replace(',', '.').astype(float)

# Pré-processamento
data['sex'] = data['sex'].map({'male': 0, 'female': 1})
data['smoker'] = data['smoker'].map({'yes': 1, 'no': 0})
data = pd.get_dummies(data, columns=['region'], drop_first=True)

# Definir variáveis independentes e dependentes
X = data[['age', 'sex', 'children', 'smoker']].values
Y = data['charges'].values

# Converter Y para array NumPy e garantir que seja uma coluna
Y = Y.reshape(-1, 1)

# Escalonamento das variáveis
mean_X = np.mean(X, axis=0)
std_X = np.std(X, axis=0)
X_scaled = (X - mean_X) / std_X

# Adicionar o intercepto
X_scaled = np.c_[np.ones(X_scaled.shape[0]), X_scaled]

# Funções auxiliares para operações matriciais
def matrix_multiply(A, B):
    return np.dot(A, B)

def transpose(A):
    return np.transpose(A)

def inverse(A):
    return np.linalg.inv(A)

# Solução analítica
XT = transpose(X_scaled)
XTX = matrix_multiply(XT, X_scaled)
XTX_inv = inverse(XTX)
XTY = matrix_multiply(XT, Y)

beta_hat = matrix_multiply(XTX_inv, XTY)

print("Vetor de parâmetros estimado (beta_hat):", beta_hat.ravel())

# Decomposição QR
def gram_schmidt(A):
    (num_rows, num_cols) = A.shape
    Q = np.zeros((num_rows, num_cols))
    R = np.zeros((num_cols, num_cols))

    for j in range(num_cols):
        v = A[:, j]
        for i in range(j):
            q = Q[:, i]
            R[i, j] = q.dot(v)
            v = v - R[i, j] * q
        R[j, j] = np.linalg.norm(v)
        Q[:, j] = v / R[j, j]

    return Q, R

Q, R = gram_schmidt(X_scaled)
QTY = matrix_multiply(transpose(Q), Y)
beta_hat_qr = matrix_multiply(inverse(R), QTY)

print("Vetor de parâmetros estimado via QR (beta_hat_qr):", beta_hat_qr.ravel())

# Função de custo (Erro Quadrático Médio)
def compute_cost(X, Y, beta):
    m = len(Y)
    J = np.sum((matrix_multiply(X, beta) - Y) ** 2) / (2 * m)
    return J

# Gradiente descendente
def gradient_descent(X, Y, beta, learning_rate, num_iterations):
    m = len(Y)
    cost_history = []

    for i in range(num_iterations):
        gradient = matrix_multiply(transpose(X), (matrix_multiply(X, beta) - Y)) / m
        beta = beta - learning_rate * gradient

        # Verificação de overflow
        if np.any(np.isnan(beta)) or np.any(np.isinf(beta)):
            print("Warning: overflow detected. Stopping gradient descent.")
            break

        cost_history.append(compute_cost(X, Y, beta))

    return beta, cost_history

# Inicialização dos parâmetros
beta = np.zeros((X_scaled.shape[1], 1))
learning_rate = 0.01  # Ajustar a taxa de aprendizado
num_iterations = 1000

# Execução do gradiente descendente
beta_hat_gd, cost_history = gradient_descent(X_scaled, Y, beta, learning_rate, num_iterations)

print("Vetor de parâmetros estimado via Gradiente Descendente (beta_hat_gd):", beta_hat_gd.ravel())

# Função de predição
def predict(X, beta):
    return matrix_multiply(X, beta)

# Predições
Y_pred_analytical = predict(X_scaled, beta_hat)
Y_pred_qr = predict(X_scaled, beta_hat_qr)
Y_pred_gd = predict(X_scaled, beta_hat_gd)

# Função de cálculo do MSE
def mean_squared_error(Y_true, Y_pred):
    return np.sum((Y_true - Y_pred) ** 2) / len(Y_true)

# Cálculo do MSE
mse_analytical = mean_squared_error(Y, Y_pred_analytical)
mse_qr = mean_squared_error(Y, Y_pred_qr)
mse_gd = mean_squared_error(Y, Y_pred_gd)

print("MSE - Solução Analítica:", mse_analytical)
print("MSE - Decomposição QR:", mse_qr)
print("MSE - Gradiente Descendente:", mse_gd)

Vetor de parâmetros estimado (beta_hat): [13943.61574115  3844.25775649  -268.15636338   357.33165539
  9777.44641836]
Vetor de parâmetros estimado via QR (beta_hat_qr): [13943.61574115  3844.25775649  -268.15636338   357.33165539
  9777.44641836]
Vetor de parâmetros estimado via Gradiente Descendente (beta_hat_gd): [13943.01377787  3844.15476092  -268.48257983   357.26909347
  9776.91431131]
MSE - Solução Analítica: 47740252.73553374
MSE - Decomposição QR: 47740252.73553373
MSE - Gradiente Descendente: 47740253.474948265


Questao 2


1. Objetivo da Modelagem de Regressão Logística
O objetivo da modelagem de regressão logística é prever a probabilidade de um evento ocorrer, com base em variáveis independentes. No caso deste conjunto de dados, o objetivo provavelmente é prever se um cliente irá adquirir um produto (por exemplo, um cartão de crédito) ou não, com base em variáveis como gênero, idade, histórico de crédito etc.

### Função de Perda (Entropia Cruzada Negativa) para Regressão Logística:

\[ \text{Loss} = -\frac{1}{n} \sum_{i=1}^{n} [y_i \log(p_i) + (1 - y_i) \log(1 - p_i)] ]\

### Derivação da Função de Perda:

Para encontrar os parâmetros \( \beta \) que maximizam a função de perda, precisamos calcular a derivada da função de perda em relação a cada parâmetro \( \beta_j \). A derivada da função de perda em relação a \( \beta_j \) é dada por:

\[ \frac{\partial \text{Loss}}{\partial \beta_j} = -\frac{1}{n} \sum_{i=1}^{n} (y_i - p_i) x_{ij} \]

onde:
- \( n \) é o número de observações
- \( y_i \) é a variável de resposta (0 ou 1)
- \( p_i \) é a probabilidade estimada da classe positiva
- \( x_{ij} \) é o valor da variável independente \( j \) para a observação \( i \)

Essa derivação é usada nas atualizações dos parâmetros no método de Newton-Raphson e no gradiente descendente para encontrar os parâmetros ótimos do modelo de regressão logística.






In [29]:
import pandas as pd

# Carregar os dados
url = 'https://raw.githubusercontent.com/Cayan-Portela/ceub/main/dados/bank_customer_treino.csv'
data = pd.read_csv(url)

# Pré-processamento
# Codificar variáveis categóricas (gender, country)
data = pd.get_dummies(data, columns=['gender', 'country'], drop_first=True)

# Dividir os dados em variáveis independentes (X) e variável dependente (y)
X = data[['credit_score', 'age', 'credit_card']].values
y = data['churn'].values


In [30]:
import numpy as np

# Adicionar o intercepto aos dados
X = np.c_[np.ones(X.shape[0]), X]

# Função logística
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Método de Newton-Raphson
def newton_raphson(X, y, max_iter=1000, tol=1e-6):
    beta = np.zeros(X.shape[1])
    m = len(y)

    for _ in range(max_iter):
        # Calcular as probabilidades previstas
        pi = sigmoid(np.dot(X, beta))

        # Calcular a matriz diagonal W
        W = np.diag(pi * (1 - pi))

        # Calcular o vetor de erros
        error = np.dot(X.T, (y - pi))

        # Calcular a matriz Hessian
        hessian = np.dot(X.T, np.dot(W, X))

        # Atualizar os parâmetros beta
        beta_old = beta
        beta = beta + np.dot(np.linalg.inv(hessian), error)

        # Critério de parada
        if np.linalg.norm(beta - beta_old) < tol:
            break

    return beta

# Estimar os parâmetros beta usando o método de Newton-Raphson
beta_newton_raphson = newton_raphson(X, y)
print("Vetor de parâmetros estimado (Newton-Raphson):", beta_newton_raphson)


Vetor de parâmetros estimado (Newton-Raphson): [-3.51023852e+00 -6.34010669e-04  6.38419769e-02 -5.35868008e-02]


In [34]:
# Gradiente Descendente
def gradient_descent(X, y, learning_rate=0.001, max_iter=1000, tol=1e-6):
    beta = np.zeros(X.shape[1])
    m = len(y)

    for _ in range(max_iter):
        # Calcular as probabilidades previstas
        pi = sigmoid(np.dot(X, beta))

        # Calcular o gradiente da função de perda
        gradient = np.dot(X.T, (y - pi))

        # Atualizar os parâmetros beta
        beta_old = beta
        beta = beta + learning_rate * gradient

        # Critério de parada
        if np.linalg.norm(beta - beta_old) < tol:
            break

    return beta

# Estimar os parâmetros beta usando o gradiente descendente
beta_gradient_descent = gradient_descent(X, y)
print("Vetor de parâmetros estimado (Gradiente Descendente):", beta_gradient_descent)


<ipython-input-30-49e55c2a0d2d>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Vetor de parâmetros estimado (Gradiente Descendente): [-1.68831613e+00 -2.31171506e+03  6.60992852e+03 -1.52108943e+01]


In [37]:
from sklearn.linear_model import LogisticRegression

# Criar o modelo de regressão logística
log_reg = LogisticRegression()

# Treinar o modelo
log_reg.fit(X, y)

# Obter os coeficientes (betas)
beta_sklearn = np.concatenate((log_reg.intercept_, log_reg.coef_[0]))

print("Vetor de parâmetros estimado (Scikit-learn):", beta_sklearn)


Vetor de parâmetros estimado (Scikit-learn): [-1.73006874e+00 -1.72144868e+00 -6.99082742e-04  6.35218949e-02
 -5.66306945e-02]


Newton Raphson para raiz de funcao


In [38]:
def newton_raphson_sqrt(n, precision=1e-10):
    # Função cuja raiz queremos encontrar: f(x) = x^2 - n
    def f(x):
        return x**2 - n

    # Derivada da função f(x): f'(x) = 2x
    def f_prime(x):
        return 2 * x

    # Estimativa inicial
    x = n / 2

    # Iterar até atingir a precisão desejada
    while True:
        # Atualizar a estimativa da raiz usando o método de Newton-Raphson
        x_next = x - f(x) / f_prime(x)

        # Verificar a condição de parada
        if abs(x_next - x) < precision:
            break

        # Atualizar a estimativa atual da raiz
        x = x_next

    return x

# Encontrar a raiz quadrada de 10 com precisão de 10 casas decimais
sqrt_10 = newton_raphson_sqrt(10)
print("Raiz quadrada de 10:", sqrt_10)


Raiz quadrada de 10: 3.1622776601683795


Achar raiz de funcao

In [49]:
def newton_raphson(f, f_prime, x0, tol=1e-10, max_iter=1000):
    """
    Função para encontrar a raiz de uma equação usando o método de Newton-Raphson.

    Args:
    - f: Função cuja raiz queremos encontrar.
    - f_prime: Derivada da função f.
    - x0: Estimativa inicial.
    - tol: Tolerância (critério de parada).
    - max_iter: Número máximo de iterações permitidas.

    Returns:
    - Aproximação da raiz da função.
    """
    x = x0
    iter_count = 0

    while True:
        x_next = x - f(x) / f_prime(x)
        iter_count += 1

        if abs(x_next - x) < tol or iter_count >= max_iter:
            break

        x = x_next

    return x

# Definir a função e sua derivada
def f(x):
    return x**4 - 2*x**3 - 2

def f_prime(x):
    return 4*x**3 - 6*x**2

# Estimativa inicial dentro do intervalo [1, 2]
x0 = 1.501

# Encontrar a raiz usando o método de Newton-Raphson
root = newton_raphson(f, f_prime, x0)

print("Raiz encontrada:", root)


Raiz encontrada: 2.1903279467148673


In [50]:
def newton_raphson_steps(f, f_prime, x0, num_steps):
    """
    Função para calcular os primeiros passos do algoritmo de Newton-Raphson.

    Args:
    - f: Função cuja raiz queremos encontrar.
    - f_prime: Derivada da função f.
    - x0: Estimativa inicial.
    - num_steps: Número de passos a serem calculados.

    Returns:
    - Lista contendo os valores de x em cada passo.
    """
    x_values = [x0]

    for _ in range(num_steps):
        x_next = x_values[-1] - f(x_values[-1]) / f_prime(x_values[-1])
        x_values.append(x_next)

    return x_values

# Definir a função e sua derivada
def f(x):
    return 3 * x**(1/3)

def f_prime(x):
    return 1 / (x**(2/3))

# Estimativa inicial
x0 = 0.1

# Número de passos
num_steps = 10

# Calcular os 10 primeiros passos do algoritmo de Newton-Raphson
steps = newton_raphson_steps(f, f_prime, x0, num_steps)

# Imprimir os resultados
for i, x in enumerate(steps):
    print(f"Passo {i}: x = {x:.10f}")


Passo 0: x = 0.1000000000
Passo 1: x = -0.2000000000
Passo 2: x = 0.4000000000-0.0000000000j
Passo 3: x = -0.8000000000+0.0000000000j
Passo 4: x = 1.6000000000-0.0000000000j
Passo 5: x = -3.2000000000+0.0000000000j
Passo 6: x = 6.4000000000-0.0000000000j
Passo 7: x = -12.8000000000+0.0000000000j
Passo 8: x = 25.6000000000-0.0000000000j
Passo 9: x = -51.2000000000+0.0000000000j
Passo 10: x = 102.4000000000-0.0000000000j


In [52]:
def newton_raphson(f, f_prime, x0, tol=1e-10, max_iter=1000):
    """
    Função para encontrar a raiz de uma equação usando o método de Newton-Raphson.

    Args:
    - f: Função cuja raiz queremos encontrar.
    - f_prime: Derivada da função f.
    - x0: Estimativa inicial.
    - tol: Tolerância (critério de parada).
    - max_iter: Número máximo de iterações permitidas.

    Returns:
    - Aproximação da raiz da função.
    """
    x = x0
    iter_count = 0

    while True:
        x_next = x - f(x) / f_prime(x)
        iter_count += 1

        if abs(x_next - x) < tol or iter_count >= max_iter:
            break

        x = x_next

    return x

# Definir a função e sua derivada
def f(x):
    return x**4 - 5*x**3 + 9*x + 3

def f_prime(x):
    return 4*x**3 - 15*x**2 + 9

# Estimativa inicial dentro do intervalo [4, 6]
x0 = 5
# Encontrar a raiz usando o método de Newton-Raphson
root = newton_raphson(f, f_prime, x0)

print("Raiz encontrada:", root)


Raiz encontrada: 4.528917957294362
